# Transforming Text into Features Using Word Embeddings

In this demo, you will see how to train a Word2Vec model to obtain word embeddings for spam message classification. We will train a logistic regression model using the embeddings to create our feature vectors. We will then use our model to predict whether a new email is spam.

A Word2Vec model should be trained on a large text corpus that is relevant to the domain in which you will later use a machine learning model to make predictions. For example, if you will be working with medical data, your Word2Vec model should not be trained on text data originating from product reviews, but rather should be trained on medical data.

You can train a Word2Vec model using an existing tool, such as Gensim, which is a Python package used for natural language processing. After the embeddings have been produced, you will use them to create feature vectors for your machine learning model. For example, we can represent each training example as a feature vector by taking the average of the Word2Vec embeddings of the words in that training example. Once we have created feature vectors for all of our training examples, we can fit our machine learning model to the training data and use our trained model to make predictions. 

This demo we will walk you through the following steps:

1. Load the spam message data set. The spam email dataset contains email subject lines.

2. Create labeled examples out of the data: We will have one text feature and one label. Each feature will contain one email subject line.

3. Preprocess the text features: We will preprocess the data by removing stop words, converting all text to lowercase, removing punctuation, etc. from the email subject lines. 

4. Create training and test datasets: Each example in the training and test datasets will now contain one preprocessed email subject line as a feature.

5. Train the Word2Vec model using the training dataset's features: we will train a Word2Vec model using Gensim. The model will figure out how to represent words as vectors based on how they are used in the spam email dataset. You will inspect the resulting word embeddings to develop a better understanding of what they can tell you about a particular word.

6. Create feature vectors out of the training and test data: After training the Word2Vec model, we will represent the features in every training and test example (recall, each example contains an email subject line) as a vector by taking the average of the Word2Vec embeddings of the words in the example. 

7. Train a logistic regression model using the feature vectors as input features, and evaluate the model's performance.

**<font color='red'>Note: Some of the code cells in this notebook may take a while to run.</font>**

### Import Packages

Before you get started, import a few packages. Run the code cell below. 

In [1]:
import pandas as pd
import numpy as np
import os 
import matplotlib as plt
import seaborn as sns

We will also import the scikit-learn `LogisticRegression class`, the `train_test_split()` function for splitting the data into training and test sets, and the `roc_auc_score()` function to evaluate the model.


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

## Step 1: Build Your DataFrame and Define Your ML Problem

We will work with a dataset containing emails that are labeled as either spam or not spam.

#### Load a Data Set and Save it as a Pandas DataFrame

In [3]:
filename = os.path.join(os.getcwd(), "data", "spamDataset.csv")
df = pd.read_csv(filename, header=0)

df.head()

,email_text,spam
0,Subject: enron methanol ; meter # : 988291\r\n...,False
1,"Subject: hpl nom for january 9 , 2001\r\n( see...",False
2,"Subject: neon retreat\r\nho ho ho , we ' re ar...",False
3,"Subject: photoshop , windows , office . cheap ...",True
4,Subject: re : indian springs\r\nthis deal is t...,False


#### Define the Label

This is a binary classification problem in which we will predict whether an email is a spam or not spam. The label is the `spam` column.

#### Identify Features

We only have one feature. The feature is the `email_text` column.


## Step 2: Create Labeled Examples from the Data Set

Let's create labeled examples from our dataset. We will have one text feature and one label. 
The code cell below carries out the following steps:

* Gets the `spam` column from DataFrame `df` and assigns it to the variable `y`. This will be our label. Note that the label contains True or False values that indicate whether an email is spam or not spam.
* Gets the column `email_text` from DataFrame `df` and assigns it to the variable `X`. This will our feature. Note that the `email_text` feature contains the subject line of the email.


In [4]:
y = df['spam'] 
X = df['email_text']

X.shape

(5170,)

In [5]:
X.head()

0    Subject: enron methanol ; meter # : 988291\r\n...
1    Subject: hpl nom for january 9 , 2001\r\n( see...
2    Subject: neon retreat\r\nho ho ho , we ' re ar...
3    Subject: photoshop , windows , office . cheap ...
4    Subject: re : indian springs\r\nthis deal is t...
Name: email_text, dtype: object

Let's take a look at an example of a spam email and an email that is not spam.

In [6]:
print('A spam email: \n\n', X[67])
print('A non-spam email: \n\n', X[135])

A spam email: 

 Subject: re : husband soup would be
as you know election time is not the best thing for the economy .
economy is in a very unstable condition , as you can see gas prices
are going up along with the m o rtgvage rat e s . once the
r a te goes up you will not have a chance to s av e money again
for a very long time .
it is your last chance . get r e f inanced at 4 . 2 point !
http : / / www . fintod . com /
- -
despoil , compote a amende
the me orbital irruption
gfawn a ax henrietta
a the in boatswain
out whither the accompanist lint macintosh

A non-spam email: 

 Subject: re : tuesday , december 26 th
i will be here tuesday , also .
mark mccoy
12 / 20 / 2000 09 : 04 am
to : michael olsen / na / enron @ enron , tom acton / corp / enron @ enron , clem
cernosek / hou / ect @ ect , robert cotten / hou / ect @ ect , jackie young / hou / ect @ ect ,
sabrae zajac / hou / ect @ ect , carlos j rodriguez / hou / ect @ ect , mark
mccoy / corp / enron @ enron , aimee lannou / hou /

## Step 3: Preprocess the Text

The next step is to preprocess the text. Preprocessing technqiues can include cleaning the data, converting all text to lowercase, removing special characters, removing stopwords from the text, tokenizing the text (split it into smaller chunks), and lemmatizing the text (converting a word to its root word).

You can perform preprocessing on your own or use tools to accomplish this. One common tool is NLTK [Natural Language Toolkit](https://www.nltk.org/). However, for this demo, we will use the built-in function from Gensim to preprocess the text. This function will remove some stop words, covert all text to lowercase, remove punctuation and tokenize the text.

Let's import the Gensim package.

In [7]:
import gensim

Before we preprocess our data, let's look at an example output of preprocessed text. Let's take a simple sentence and perform preprocessing. Run the code cell below and inspect the results.

In [8]:
sentence = "I went to the market to buy some apples for my pet horse." 

list(gensim.utils.simple_preprocess(sentence))

['went',
 'to',
 'the',
 'market',
 'to',
 'buy',
 'some',
 'apples',
 'for',
 'my',
 'pet',
 'horse']

Now let's perform preprocessing on our `email_text` feature and compare the difference between the original text and the preprocessed text

In [9]:
original_X = X
X = X.apply(lambda row: gensim.utils.simple_preprocess(row))

In [10]:
X.head()

0    [subject, enron, methanol, meter, this, is, fo...
1    [subject, hpl, nom, for, january, see, attache...
2    [subject, neon, retreat, ho, ho, ho, we, re, a...
3    [subject, photoshop, windows, office, cheap, m...
4    [subject, re, indian, springs, this, deal, is,...
Name: email_text, dtype: object

In [11]:
original_X.head()

0    Subject: enron methanol ; meter # : 988291\r\n...
1    Subject: hpl nom for january 9 , 2001\r\n( see...
2    Subject: neon retreat\r\nho ho ho , we ' re ar...
3    Subject: photoshop , windows , office . cheap ...
4    Subject: re : indian springs\r\nthis deal is t...
Name: email_text, dtype: object

## Step 4: Create Training and Test Data Sets

Let's take our preprocessed text data set and split the data into training and test sets with 80% of the data being the training set.

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.80, random_state=1234)

X_train.head()

562     [subject, hpl, nom, for, sept, see, attached, ...
148     [subject, copanno, changes, forwarded, by, ami...
4831    [subject, re, maynard, oil, revised, nom, dare...
3385    [subject, deal, for, december, can, either, of...
2389    [subject, heisse, sx, action, hallo, mein, lie...
Name: email_text, dtype: object

## Step 5: Training the Word2Vec Model and Inspecting the Word Embeddings

Now that the data has been preprocessed and we have our training data, we will train a Word2Vec model using Gensim using the training data `X_train`. For more information on Gensim, consult the online [documentation](https://radimrehurek.com/gensim/models/word2vec.html). The model will produce word embeddings. That is, words represented as numerical vectors based on how these words are used in the email spam dataset.
    
We will specify the following parameters:

* size: dimension of each resulting vector that will represent each word.
* window: the number of words behind or ahead of a target word that will be used to provide context for that word
* min_count: the number of times a word must appear in our text document in order to create a word vector. The model will ignore words that do not satisfy the `min_count` specification, therefore ignoring wors that are not important.



In [13]:
print("Begin")
word2vec_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

print("End")

Begin
End


You can check the size of a particular model with the `len()` method. 

In [14]:
len(word2vec_model.wv.key_to_index)  # retrieve vocabulary and measure its size

18015

While 18,015 words may seem like a lot, this is actually quite small for most real-world applications. In practice, you can use models with larger vector sizes (say 300) and much larger vocabulary.

To check if a word is included the vocabulary, you can use the `key_to_index` attribute. 

In [15]:
'cornell' in word2vec_model.wv.key_to_index

False

In [16]:
'dog' in word2vec_model.wv.key_to_index

True

The word `'dog'` is in our vocabulary. Let's inspect its word vector. The code cell belows retrieves the vector for the word `'dog'`. 

In [17]:
word2vec_model.wv['dog']

array([-0.10418516,  0.28754872,  0.0032218 , -0.16407667,  0.00459164,
       -0.16941477,  0.1187093 ,  0.40616184, -0.24570939, -0.17889339,
       -0.20599638, -0.31346825, -0.01501046, -0.07445442,  0.17699915,
       -0.28091705,  0.12673515, -0.16082834, -0.00717387, -0.3441623 ,
        0.11102255,  0.07512208,  0.18799382, -0.18664314, -0.00654916,
        0.08451083, -0.19960502, -0.11766133, -0.23625924,  0.00834361,
        0.17555152, -0.07461119,  0.07405424, -0.26532254, -0.1267233 ,
        0.09531993, -0.10134591, -0.05699705, -0.14955416, -0.2722893 ,
        0.06493951, -0.0759764 , -0.1773903 ,  0.06351222,  0.17764902,
       -0.16241336, -0.00512158, -0.19012223,  0.07147191,  0.23980543,
        0.05089482, -0.11514437, -0.23020844, -0.08149247, -0.22013262,
        0.01252646, -0.02097434, -0.02906141, -0.22805557,  0.01895173,
        0.04710523,  0.01917572,  0.10933578,  0.00967102, -0.22837038,
        0.15116575,  0.20152389,  0.17315505, -0.28032994,  0.17

Examine the vector, noting each of the following important characteristics: 

1. It contains 100 values somewhere between -0.39333 and 0.4495
2. There are no zeros, so it can be considered a **dense vector** (a vector comprised of mostly non-zero values)
3. Each value represents a dimension, but is not necessarily interpretable by humans
4. Larger (in magnitude) values may relate to categories in which `'dog'` has a presence
5. Many broader categories are also likely to be represented by a combination of these dimensions
6. A few values are relatively close to zero. This implies that these dimensions do not carry "significant information" about `'dog'`


Let's find the most similar words for `'dog'`.


In [18]:
word2vec_model.wv.most_similar('dog')

[('flight', 0.9927213788032532),
 ('nw', 0.992231011390686),
 ('grease', 0.9917516708374023),
 ('chaplain', 0.9916641116142273),
 ('baltic', 0.9915153980255127),
 ('monetary', 0.9915044903755188),
 ('catkin', 0.9914748072624207),
 ('pet', 0.9913700222969055),
 ('hymn', 0.9911980032920837),
 ('agouti', 0.9909760355949402)]

The code cell below outputs the first few 25 words that our Word2Vec model learned a vector for.

In [19]:
top25 = word2vec_model.wv.index_to_key[:25]
top25

['the',
 'to',
 'ect',
 'and',
 'for',
 'of',
 'you',
 'subject',
 'in',
 'on',
 'is',
 'this',
 'hou',
 'enron',
 'be',
 'that',
 'we',
 'from',
 'your',
 'will',
 'have',
 'with',
 'at',
 'are',
 'it']

You can go a step further and print vectors associated with each of these words. Use the `background_gradient` method to wrap these vectors into a DataFrame as rows with the corresponding words as row indices. Let's spice it up with colors.

In [20]:
pd.DataFrame({w:word2vec_model.wv[w] for w in top25}).T.style.background_gradient(cmap='coolwarm').set_precision(2)

Note that the 100 columns you see here are dimensions you specified earlier. We do not know what each dimension signifies, although this is an active area of research. However, if you examine each dimension you'll notice more red in some columns and more blue in others. Some of the words are very common and might be stop words we would normally remove. However, if we load more specific groups of words, we might make good guesses about what some of the dimensions mean.

The code cells below compute the similarities between two words in the vocabulary.

In [21]:
word2vec_model.wv.similarity(w1='dog', w2='person')

0.3981818

In [22]:
word2vec_model.wv.similarity(w1='dog', w2='cat')

0.97688043

## Step 6: Create Feature Vectors out of Word Embeddings for a Classifier

Now let's convert the features in our training and test datasets into feature vectors using our word embeddings. We will use this  to train a logistic regression model. Let's first inspect our original training and test datasets:

In [23]:
X_train.head()

562     [subject, hpl, nom, for, sept, see, attached, ...
148     [subject, copanno, changes, forwarded, by, ami...
4831    [subject, re, maynard, oil, revised, nom, dare...
3385    [subject, deal, for, december, can, either, of...
2389    [subject, heisse, sx, action, hallo, mein, lie...
Name: email_text, dtype: object

In [24]:
X_test.head()

5075    [subject, hpl, nom, for, may, see, attached, f...
3817    [subject, cleburne, tenaska, iv, plant, daren,...
2967    [subject, re, producer, connects, on, the, con...
3014    [subject, mail, hey, daren, is, this, your, ma...
2185    [subject, revision, forest, oil, november, gas...
Name: email_text, dtype: object

The code cell below walks through every example in both the training and test datasets and replaces every word contained in the `email_text` feature with its corresponding word embedding. Original words that do not have a corresponding word embedding will not be part of the training and test sets. For example, stop words that were removed when creating the word embeddings will not appear in the training and test sets.

<b>Note</b>: This may take a while to run.

In [25]:
words = set(word2vec_model.wv.index_to_key)

print('Begin transforming X_train')
X_train_word_embeddings = np.array([np.array([word2vec_model.wv[word] for word in words if word in training_example])
                        for training_example in X_train], dtype=object)
print('Finish transforming X_train')

print('Begin transforming X_test')
X_test_word_embeddings = np.array([np.array([word2vec_model.wv[word] for word in words if word in training_example])
                        for training_example in X_test], dtype=object)
print('Finish transforming X_test')


Begin transforming X_train
Finish transforming X_train
Begin transforming X_test
Finish transforming X_test


In [26]:
print('Number of words in first training example: {0}'.format(len(X_train.iloc[0])))
print('First word in first training example: {0}'.format(X_train.iloc[0][0]))
print('Second word in first training example: {0}\n'.format(X_train.iloc[0][1]))

print('Number of word vectors in first training example: {0}'.format(len(X_train_word_embeddings[0])))
print('First word vector in first training example:\n {0}'.format(X_train_word_embeddings[0][0]))
print('\nSecond word vector in first training example: \n {0}\n'.format(X_train_word_embeddings[0][1]))


Number of words in first training example: 12
First word in first training example: subject
Second word in first training example: hpl

Number of word vectors in first training example: 10
First word vector in first training example:
 [-2.209755   -1.2015774  -1.7984903  -0.6512364   0.64314634 -0.93149626
  0.53938836  1.4608094   0.75386125 -0.15215431  0.0155459  -1.0283413
 -0.27638352 -1.5788465  -1.3615109   0.8431128  -0.81761724 -0.2627952
  1.5270276   0.11079045  2.7981584   1.5826668  -0.24077404  0.2104903
  0.23325156  1.0327744  -1.9860578  -1.3389406  -1.2421668  -0.07193656
 -2.6843843   2.0337977   1.3858063  -0.31496707  2.5750449   3.2397578
 -0.9102774  -0.14159591  1.2166148  -1.1217163  -0.507891    0.96387535
 -1.1491107   0.47700706  2.538793   -2.5684855  -0.18157059 -0.5348133
  1.6737889   0.5612847   1.0434594   0.42247537  0.16265397  1.0993551
  1.4148908   2.0735295   2.148272    0.09939633 -1.2963827   0.51542825
 -1.5820359   1.035259   -0.5829752  -1.1

After replacing the `email_text` feature in our training and test data with word embeddings, each example in our training and test data now has a different number of features, each corresponding to a word vector:

In [27]:
print('Number of word vectors in first five examples in training set:')
for w in range(0, 5):
    print(len(X_train_word_embeddings[w]))

print('Number of word vectors in first five examples in test set:')
for w in range(0, 5):
    print(len(X_test_word_embeddings[w]))

Number of word vectors in first five examples in training set:
10
23
37
57
30
Number of word vectors in first five examples in test set:
10
31
59
8
59


This will cause an error when we train our model. We have to create feature vectors that will provide our classifier with a consistent set of features per example.

We can take an element-wise average of the word embeddings of the words contained in each training and test example. This makes feature vector representations that can be used as training and test features for our classifier. 

In [28]:
X_train_feature_vector = []
for w in X_train_word_embeddings:
    if w.size:
        X_train_feature_vector.append(w.mean(axis=0))
    else:
        X_train_feature_vector.append(np.zeros(100, dtype=float))
        
X_test_feature_vector = []
for w in X_test_word_embeddings:
    if w.size:
        X_test_feature_vector.append(w.mean(axis=0))
    else:
        X_test_feature_vector.append(np.zeros(100, dtype=float))

Each example now consists of one feature, which is a numerical feature vector of length 100. Run the code cell below to inspect the first five training examples.

In [29]:
for w in range(0, 5):
    print('Length of training example {0}: {1}'.format(w, len(X_train_feature_vector[w])))
    
print('First training example\'s feature vector: \n{0}'.format(X_train_feature_vector[0]))

Length of training example 0: 100
Length of training example 1: 100
Length of training example 2: 100
Length of training example 3: 100
Length of training example 4: 100
First training example's feature vector: 
[-1.2157443  -0.11076333 -0.6710328   0.02498029 -0.20808236 -0.2970723
  0.06699034  0.7648239   0.10506678 -0.75636464 -0.08489671 -0.55251795
 -0.45780635 -0.6019805  -0.43199787  0.39780268 -0.02704841  0.07874871
  0.75716436 -0.9077648   1.7157418   0.6761495   0.31093323 -0.31468886
 -0.16615328  0.64454216 -0.8371704  -0.58226836 -0.5722841  -0.00218162
 -1.0497805   1.4098011   0.8781974  -0.7006148   1.4930178   1.1674943
 -0.5432955   0.48836786  0.09580327 -1.1345088   0.13649829  0.08992314
 -1.0939554   0.22769694  1.5014294  -1.8812262  -0.12131397 -0.6004549
  0.9671831   0.36238083  0.45867187 -0.10516739  0.2507924   0.5004188
  0.49642992  1.3087094   0.8217107   0.10147096 -1.0473254   0.11855948
 -0.24564473  0.6409472  -0.0709145  -0.45379892 -1.5024436  -

## Step 7: Fit a Logistic Regression Model to the Training Data and Evaluate the Model

Now we can train our model on our transformed data. The code cell below trains a logistic regression model and computes the AUC on the test set.

In [30]:
# 1. Create a LogisticRegression model object, and fit a Logistic Regression model to the transformed training data
model = LogisticRegression(max_iter=200)
model.fit(X_train_feature_vector, y_train)

# 2. Make predictions on the transformed test data using the predict_proba() method and 
# save the values of the second column
probability_predictions = model.predict_proba(X_test_feature_vector)[:,1]

# 3. Make predictions on the transformed test data using the predict() method 
class_label_predictions = model.predict(X_test_feature_vector)

# 4. Compute the Area Under the ROC curve (AUC) for the test data. Note that this time we are using one 
# function 'roc_auc_score()' to compute the auc rather than using both 'roc_curve()' and 'auc()' as we have 
# done in the past
auc = roc_auc_score(y_test, probability_predictions)
print('AUC on the test data: {:.4f}'.format(auc))

AUC on the test data: 0.9880


Let's check two emails and see if our model properly predicted whether an email is spam or not spam.

In [31]:
print('Email #1:\n')
print(original_X[14])

print('\nPrediction: Is this a spam email? {}\n'.format(class_label_predictions[14])) 

print('Actual: Is this a spam email? {}\n'.format(y_test.to_numpy()[14]))


Email #1:

Subject: tenaska iv july
darren :
please remove the price on the tenaska iv sale , deal 384258 , for july and enter the demand fee . the amount should be $ 3 , 902 , 687 . 50 .
thanks ,
megan

Prediction: Is this a spam email? True

Actual: Is this a spam email? True



In [32]:
print('Email #2:\n')
print(original_X[132])

print('\nPrediction: Is this a spam email? {}\n'.format(class_label_predictions[132])) 

print('Actual: Is this a good spam email? {}\n'.format(y_test.to_numpy()[132]))

Email #2:

Subject: re : noms / actual flow for 3 / 29 / 01
we agree with the nomination .
" eileen ponton " on 03 / 30 / 2001 10 : 05 : 40 am
to : david avila / lsp / enserch / us @ tu , charlie stone / texas utilities @ tu , melissa
jones / texas utilities @ tu , hpl . scheduling @ enron . com ,
liz . bellamy @ enron . com
cc :
subject : noms / actual flow for 3 / 29 / 01
nom mcf mmbtu
24 , 583 24 , 999 25 , 674
btu = 1 . 027

Prediction: Is this a spam email? False

Actual: Is this a good spam email? False

